# INTRODUÇÃO

A música está presente em todas as culturas, povos e tribos sendo considerada uma das principais artes do mundo e muito relacionada as emoções e sentimentos como tristeza, alegria, raiva e diversos movimentos sociais se definem por suas músicas.
Independente da língua utilizada consegue transmitir sentimentos e emoções, diferentes áreas do cerebro são estimuladas a partir de ritmos musicais sejam com batidas fortes ou regulares e lentos com ativação cerebral como nenhum outro estimulo conhecido, alguns benefícios relacionados a efeitos positivos como fortalecimento do aprendizado, regulação de hormônios do extress, pressão arterial e pulsação. 

A cada ano o Spotify seleciona as músicas mais ouvidas e faz a compilação em uma lista de reprodução das músicas mais transmitidas ao longo do ano. 
No presente estudo, utilizou-se a lista de reprodução do ano 2018, o que as músicas têm em comum? Porque as pessoas gostam delas?

Como fonte de dados foi utilizado o arquivo .csv (Top2018.csv) e inclui nome da música, artista da música, recursos de áudio como: dança, andamento, instrumentalidade, vivacidade, acústica, duração, tempo entre outras variáveis.

Selecionadas as variáveis quantitativas: tempo, duração_ms, valence, danceability, acousticness, key, speechiness, liveness com as quais foram avaliadas o tempo, a vivacidade, danceabilidade, sonoridade das músicas mais ouvidas em 2018 e a relação entre elas que podem ter influenciado na escolha por parte dos usuários.

Realizou-se a análise com recurso a frequências absolutas e relativas e a medidas de tendência central como a média, mediana, máximo, mínimo e o Desvio Padrão, regressão linear entre outras análises para determinar padrões e perfis para escolha das músicas.

Utilizou-se como parametro para as análises o nível de significância de 0.5%.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Importando as Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import matplotlib.ticker as ticker
from sklearn import datasets, linear_model
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

# Apresentação dos dados

In [ ]:
# Importando e apresentando o arquivo
arquivo = pd.read_csv('/kaggle/input/top-spotify-tracks-of-2018/top2018.csv', encoding='utf-8')
arquivo.head()

In [ ]:
arquivo.info()

In [ ]:
arquivo.corr()

A partir das variáveis apresentadas acima, identificou-se que as variáveis danceability, energy, loudness e tempo são que possuem melhor correlação.

In [ ]:
# Histograma com a distribuição das variáveis
arquivo.hist()
plt.show()

# Correlação entre as variáveis

In [ ]:
#Plotar um gráfico Heatmap com a correlação entre as variáveis. 

plt.suptitle("Gráfico - Correlação entre variáveis")
sns.heatmap(arquivo.corr(),cmap='YlOrRd')
plt.show()

In [ ]:
Correlacao=arquivo[['danceability','energy','valence','loudness','tempo']]
Correlacao

In [ ]:
#Plotar um gráfico Heatmap com a correlação mais fortes entre as variáveis. 

plt.suptitle("Gráfico - Correlações mais fortes")
sns.heatmap(Correlacao.corr(), annot=True, cmap="viridis")
plt.show()

In [ ]:
#Gráfico Joinplot usando como base os dados correlacionados e uma regressão linear entre as variáveis energy e loudness. 

sns.jointplot(data=Correlacao,y='energy',x='loudness',kind='reg', color='r')

# Análise Descritiva das variáveis:
Tabela com a análise de média, desvio padrão, valores min e max e também o primeiro, segundo e terceiro quartis.

In [ ]:
#Análise descritiva
arquivo.describe().T

# Duração das Músicas 

In [ ]:
# A duração das músicas.
arquivo['Duration_min']=arquivo['duration_ms']/60000
print(arquivo)

In [ ]:
#Gráfico com a distribuição da duração das músicas em milisegundos
plt.figure(figsize=(6,4))
sns.swarmplot(arquivo['duration_ms'], color='r')

È possível identificar que a maior concentração da duração das musicas encontra-se entre 150000 e 250000 milisegundos.

# Artistas 

In [ ]:
plt.suptitle("Gráfico - Identificação dos artistas")
sns.barplot(data=arquivo.sort_values(by='key', ascending=False).head(15), x='key', y='artists');

In [ ]:
#Postando os 10 primeiros artistas na lista das músicas mais tocadas:
arquivo['artists'].value_counts().head(10)

Na lista acima, constata-se que os 3 principais artistas são Post Malone, XXXTentacion com 6 músicas cada e Drake com 4 músicas, em seguida vamos identificar as músicas e os dados em relação as mesmas.

In [ ]:
#Artista 1 = Post Malone
art1 = arquivo[arquivo['artists']=='Post Malone']
art1[['name','danceability','energy','loudness','valence','tempo']]

In [ ]:
#Artista 2 = XXXTENTACION
art2 = arquivo[arquivo['artists']=='XXXTENTACION']
art2[['name','danceability','energy','loudness','valence','tempo']]

In [ ]:
#Artista3 = Drake
art3 = arquivo[arquivo['artists']=='Drake']
art3[['name','danceability','energy','loudness','valence','tempo']]

# Danceabilidade das músicas

In [ ]:
# A um nível de significância de 0.5% quais as músicas mais dançantes:
mu=arquivo['danceability']>=0.75
re=(arquivo['danceability']>=0.5) & (arquivo['danceability']<0.75)
nd=arquivo['danceability']<0.5

data=[mu.sum(),re.sum(),nd.sum()]
Dance=pd.DataFrame(data,columns=['percent'],
                   index=['Muito','Regular','Ñ Dançante/Instrumental'])
Dance

In [ ]:
#Plotando gráfico de distribuição de danceabilidade
plt.suptitle("Gráfico - Distribuição de Danceabilidade das músicas")
sns.distplot(arquivo['danceability'], color='y')

In [ ]:
#Regressão linear entre danceabilidade e Valencia das músicas
x1 = arquivo["danceability"].values
y2 = arquivo["valence"].values

x = x1.reshape(x1.shape[0], 1)
y = y2.reshape(y2.shape[0], 1)

regressao = linear_model.LinearRegression()
regressao.fit(x, y)

fig= plt.figure(figsize=(7,5))
plt.suptitle("Correlação entre as variáveis Danceability e Valence")

ax = plt.subplot(1,1,1)
ax = plt.scatter(x1, y2, alpha=0.5, color='blue')
ax = plt.plot(x1, regressao.predict(x), color='red', linewidth=2)
plt.xticks(())
plt.yticks(())

plt.xlabel("danceability")
plt.ylabel("valence")

plt.show()

  Esta análise trouxe as informações das músicas consideradas mais dançantes a um nível de significância de 0.5%, para melhor compreensão foram divididas em categorias:
    
   Maior que 75% -> Muito dançante;
   
   Entre 50% e 74% -> Regular;
   
   Menor que 50% -> Não dançante/Música Instrumental

In [ ]:
#Criar tabela com as 10 músicas mais dançantes
arquivo[['name','artists','danceability','valence','tempo']].sort_values(by='danceability', ascending=False).head(10)

Tabela com as 10 músicas mais dançantes entre as 100 selecionadas

# Intensidade de energia

In [ ]:
#Montando tabela com análise de intensidade de energia a partir do nível de significância de 0.5%
mu=arquivo['energy']>=0.75
re=(arquivo['energy']>=0.5) & (arquivo['energy']<0.75)
nd=arquivo['energy']<0.5

data=[mu.sum(),re.sum(),nd.sum()]
energy=pd.DataFrame(data,columns=['Percentual'],
                   index=['Alta','Regular','Pouca energia'])
energy

In [ ]:
#Plotando gráfico de distribuição de energia
plt.suptitle("Gráfico - Distribuição de energia")
sns.distplot(arquivo['energy'])

A um nível de significância de 0.5% é possível observar que 55% das músicas são classificadas como regulares e com distribuição entre 0.6 e 0.8 no gráfico acima. 

In [ ]:
#Criar tabela com as 10 músicas maior energia
arquivo[['name','artists','energy','valence','tempo']].sort_values(by='energy', ascending=False).head(10)

In [ ]:
#Gráfico Scatterplot com indicando a correlação e distribuição das variáveis.
plt.figure(figsize=(6,5))
arquivo.plot(title = 'Análise danceabilidade e energia de uma musica', kind='scatter', x='danceability',y='energy',color='red')
plt.figure(figsize=(6,5))
arquivo.plot(title = 'Análise', kind='scatter', x='loudness',y='speechiness',color='g')
plt.figure(figsize=(6,5))
arquivo.plot(title = 'Análise', kind='scatter', x='valence',y='liveness',color='b') 

# Ànalise de tempo/Ritmo

Os ritmos musicais são classificados a partir de escalas de andamento, grau de compasso, tempo ou até mesmo marca em partituras. 
As musicas são gravadas usando o metrômetro um aparelho de relojoaria que ajuda a manter o padrão de batidas de cada andamento musical, o metrômetro, que utilizando os bpm's avalia a pulsação constante.

Os andamentos são classificados em: 

Andante (63 – 72bpm): andamento pausado como de quem passeia; 

Adagio (54 – 58bpm): calmo;

Lento (50 – 54bpm): devagar;

Presto - (180 - 200bpm)  extremamente rápido;

Andante -  (75 - 107bpm)  Em ritmo do andar humano, agradável e compassado;

In [ ]:
# Criar um classificação entre os tempos das músicas para definir os ritmos.

arquivo.loc[arquivo['tempo']>168,'Ritmo']='Presto'
arquivo.loc[(arquivo['tempo']>=110) & (arquivo['tempo']<=168),'Ritmo']='Allegro'
arquivo.loc[(arquivo['tempo']>=76) & (arquivo['tempo']<=108),'Ritmo']='Andante'
arquivo.loc[(arquivo['tempo']>=66) & (arquivo['tempo']<=76),'Ritmo']='Adagio'
arquivo.loc[arquivo['tempo']<65,'Ritmo']='Lento'

arquivo['Ritmo'].value_counts()

Os ritmos identificados são: Allegro, Andante, Presto, Adagio e Lento e estão relacionados com os gêneros musicais: Hip Hop, Reggea, Pop.

In [ ]:
sns.set_style(style='dark')
rit1 = arquivo['Ritmo'].value_counts()
rit2 = pd.DataFrame(rit1)

sns.barplot(x=rit1, y=rit2.index, data=arquivo)
plt.title("Gráfico - Classificação do Andamento musical")
plt.show()

Por meio do gráfico identificou-se que os ritmos allegro e Andante foram os que tiveram maior ocorrência com mais de 40 musicas. 

# Conclusão
As pessoas fazem escolhas de músicas de acordo com a reação que a mesma desperta e está ligada a sentimentos, prazer, satisfação e também com a influência biológica como a liberação de hormônios no corpo e a atividade neural despertada por estímulos presente em todo o cerébro. 

Por meio das análises acima pudemos concluir que a escolha de uma música tem relação com a energia, o tempo de duração, a danceabilidade, sonoridade, vivacidade, logo para que uma determinada música seja escolhida e esteja entre as mais ouvidas o ritmo, o tempo, a valencia a energia transmitidos por uma música precisam estar presentes. 

Empresas como Spotify tem investido e cada dia diversificado seu conteúdo a partir de pesquisas e estudos de dados sobre a escolha, tipos de músicas e seus atributos, desta forma assim como vem acontecendo com outros setores os estudos e análises de dados para predição, melhoria na oferta e na produção de produtos e conteúdo, entendimento do comportamento, das escolhas futuras vem agregar ao mercado da música e também ao consumo pela população.


“A vida é para quem é corajoso o suficiente para se arriscar e humilde o bastante para aprender.”
Clarisse Lispector